In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import os.path
import time, itertools, re
import importlib
from collections import Counter
import scipy as sc
import miceforest as mf
from sklearn.metrics import make_scorer, balanced_accuracy_score, confusion_matrix, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, LassoCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from EDdataset_GGZ import colsTypeCast
from HypOpt import lowest_cwacc
full_linked_data_fname='%s/maskedDAIsy_MainDec_EDEQ_SQ48_MHCSF_Honos_Lav_Visit1.tsv'%(fileloc_data)
df_all_combo=pd.read_csv(full_linked_data_fname, sep='\t', decimal=',')
df_adapted_combo, colsExtracted, subscales=colsTypeCast(df_all_combo)
adapted_combo_cols=np.setdiff1d(colsExtracted,
        ['ED_Codes','EDtype', 'SQ48-Score', 'MHCSF-Score', 'Lav-Score', 'EDEQ-Score']+list(subscales['Honos']))

print(adapted_combo_cols)

/home/sreejita/anaconda3/envs/py_env1/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/
['DT-BMI' 'DT-Disorder_Duration_Yrs' 'DT-IND_BDL_CMD' 'DT-IND_OCD_CMD'
 'DT-IND_depression_CMD' 'DT-IND_others' 'DT-IND_prev_spec_int_wo_eff'
 'DT-num_prev_routes' 'EDEQ-bodyshape' 'EDEQ-eating' 'EDEQ-weight'
 'Lav-Dissatisfaction_body' 'Lav-Negative_appraisal_body'
 'Lav-Unfamiliarity_with_body' 'MHCSF-Emotional_Well-being'
 'MHCSF-Psychological_Well-being' 'MHCSF-Social_Well-being' 'Main-Age'
 'Main-Biosex' 'Main-Education' 'SQ48-Agoraphobia' 'SQ48-Anxiety'
 'SQ48-Cognitive_Complaints' 'SQ48-Depression' 'SQ48-Hostility'
 'SQ48-Social_phobia' 'SQ48-Somatic_Complaints' 'SQ48-Vitality'
 'SQ48-Work_related_complaints']


In [2]:
%load_ext autoreload

# Classification

In [4]:

df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test']#.to_numpy()

nan_mean=np.nanmean(df_train_adapted.to_numpy(), axis=0)
nan_std=np.nanstd(df_train_adapted.to_numpy(), axis=0)
z_train_explore_nan=((df_train_adapted.to_numpy()-nan_mean)/nan_std)
z_test_df=pd.DataFrame(data=((df_test_adapted.to_numpy()-nan_mean)/nan_std), columns=adapted_combo_cols)
z_train_df=pd.DataFrame(data=z_train_explore_nan, columns=adapted_combo_cols)


kernel_mean_match = mf.ImputationKernel(data=z_train_df,num_datasets=1,mean_match_candidates=5)
kernel_mean_match.mice(10)
z_train_explore=pd.DataFrame(data=kernel_mean_match.complete_data(), columns=adapted_combo_cols)
temp_test=kernel_mean_match.impute_new_data(z_test_df)
z_test_explore=pd.DataFrame(data=temp_test.complete_data(), columns=adapted_combo_cols)
print(z_train_explore.shape, z_test_explore.shape)

(97, 29) (35, 29)


In [8]:
%autoreload 2
from HypOpt import GridSearchClassifiers, GridSearch_LVQ
sampling_strategy='not majority'
param_grid={'RF_Max_Features': [7,10, 20,25,29], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA'}
#param_grid={'RF':param_grid_rf}
GridSearchClassifiers(z_train_explore, YTrain,sampling_strategy, param_grid)

RF: 
    n_Trees  Max_Features  Lowest_CWA
3      100            10    0.606566
2      500             7    0.601768
5      500            10    0.597222
1      300             7    0.592929
KNN: 
    K Dist Metric  Lowest_CWA
3  3      cosine    0.522475
4  5      cosine    0.470960
0  3   minkowski    0.444697
5  7      cosine    0.409343
LDA: 
   Solver  Lowest_CWA
2  eigen    0.554293
0    svd    0.521970
1   lsqr    0.521970
3  eigen         NaN
Linear SVM: 
        C  Lowest_CWA
3    1.0    0.608586
4   10.0    0.596970
5  100.0    0.596970
2    0.1    0.556818
RBF SVM: 
         C  Gamma  Lowest_CWA
32  100.0  0.010    0.585859
26   10.0  0.010    0.582323
31  100.0  0.001    0.566414
25   10.0  0.001    0.497727


In [6]:
from imblearn.over_sampling import SMOTE
sampling_strategy='not majority'
labs=np.unique(YTrain)
sm=SMOTE(sampling_strategy=sampling_strategy)
X,Y=sm.fit_resample(z_train_explore, YTrain)

In [27]:
param_grid_lvq={'prot_choice': [np.array([3,3,3,1,1]),np.array([3,2,2,1,1]),
            np.array([2,2,2,1,1]),np.array([1,1,1,1,1]), np.array([1,1,2,1,1])],
               'n_comp':[7,10, 20,25,29], 'scorer_name':'Lowest_CWA'}
GridSearch_LVQ(z_train_explore, YTrain,sampling_strategy, param_grid_lvq)

GMLVQ: 
         Prot per cls Solver  Num Component Activation type  Lowest_CWA
134  [3, 2, 2, 1, 1]  lbfgs             25           swish    0.580808
122  [3, 2, 2, 1, 1]  lbfgs              7           swish    0.579293
118  [3, 3, 3, 1, 1]  lbfgs             29           swish    0.577778
30   [3, 2, 2, 1, 1]  lbfgs             20        identity    0.574747


#### Final Classication with optimal hyperparameters

In [7]:
iters=5
l_num_prots, g_num_prots=np.array([1, 1, 2, 1, 1]), np.array([3,3,3, 1, 1])
pipeRF=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", min_samples_leaf=5, 
                        class_weight='balanced_subsample',n_estimators=100, max_features=10))])
pipeKNN=Pipeline(steps=[('KNN', KNeighborsClassifier(n_neighbors=3,metric='cosine'))])
pipeLDA=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'))])
pipeQDA=Pipeline(steps=[('QDA', QuadraticDiscriminantAnalysis(reg_param=0.01))])
pipeLSVC=Pipeline(steps=[('LinSVC', SVC(kernel="linear", C=1))])
pipeRSVC=Pipeline(steps=[('RbfSVC', SVC(kernel='rbf', C=100, gamma=0.01))])
pipeLGMLVQ=Pipeline(steps=[('LGMLVQ', LGMLVQ(distance_type='local-adaptive-squared-euclidean', 
        prototype_n_per_class=l_num_prots, solver_type='lbfgs',activation_type='swish',
        relevance_n_components=7, relevance_localization="class"))])
pipeGMLVQ=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', 
            prototype_n_per_class=np.array([3, 2, 2, 1, 1]), relevance_regularization=0.001,
            activation_type='swish',relevance_n_components=25, solver_type='lbfgs'))])
pipeGNB=Pipeline(steps=[('GNB', GaussianNB())])
pipeLogLASSO=Pipeline(steps=[('LASSO',LogisticRegressionCV(Cs=[0.0001,0.001,0.01,0.1,1], solver='saga',
                    class_weight='balanced', cv=5,penalty='l1', scoring=make_scorer(lowest_cwacc)))])
pipeClassifiers={'RF':pipeRF, 'KNN':pipeKNN, 'LDA':pipeLDA, 'QDA': pipeQDA, 'LSVC': pipeLSVC, 
                 'RSVC': pipeRSVC, 'LGMLVQ': pipeLGMLVQ, 'GMLVQ': pipeGMLVQ, 'GNB': pipeGNB,
                'LogLASSO':pipeLogLASSO}
nclasses=len(np.unique(YTrain))
labs=np.unique(YTrain)
train_conf_all, test_conf_all, classifiers_all={},{},{}

In [8]:
num_prototypes=np.sum(g_num_prots)
num_features, iters=df_train_adapted.shape[1],5
all_relevance, all_prots=np.zeros((nclasses, num_features, iters)), np.zeros((num_prototypes, num_features, iters))

for key, clf in pipeClassifiers.items():
    tr_cmap=np.zeros((nclasses, nclasses, iters))
    te_cmap=np.zeros((nclasses, nclasses, iters))
    clf_all=[]
    for iter in range(iters):
        if (key=='GMLVQ') | (key=='LGMLVQ'):
            clf.fit(X, Y)
        else:
            clf.fit(X, Y)
        clf_all.append(clf)
        train_pred_labs=clf.predict(z_train_explore)
        test_pred_labs=clf.predict(z_test_explore)
        tr_cmap[:,:,iter]=confusion_matrix(YTrain, train_pred_labs, normalize='true', labels=labs)
        te_cmap[:,:,iter]=confusion_matrix(YTest, test_pred_labs, normalize='true', labels=labs)
    classifiers_all[key]=clf_all
    train_conf_all[key], test_conf_all[key]=tr_cmap, te_cmap
    cmap_dict={'Training': tr_cmap, 'Test':te_cmap}
    for split_set in ['Training', 'Test']:
        print(key+'-'+split_set)    
        if split_set=='Training':
            printlabs=[]
            for lab in labs:
                if (lab==labs[2]) | (lab==labs[4]):
                    printlabs.append('    '+lab)
                else:
                    printlabs.append(lab)
            print(printlabs)
        mean_glob, std_glob=cmap_dict[split_set].mean(axis=2), cmap_dict[split_set].std(axis=2)
        disp_num_glob=np.reshape(np.stack((mean_glob, std_glob)).ravel('F'),(nclasses,2*nclasses))
        #disp_num_glob=[disp_num_glob]
        for idx in range(nclasses):
            cmap_cell=[]
            for jdx in range(nclasses):
                cmap_cell.append('%.3f (%.3f) '%(mean_glob[idx,jdx], std_glob[idx,jdx]))
            print(cmap_cell)    

RF-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED', '    Others']
['0.926 (0.014) ', '0.029 (0.000) ', '0.000 (0.000) ', '0.046 (0.014) ', '0.000 (0.000) ']
['0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.015 (0.031) ', '0.077 (0.000) ', '0.892 (0.038) ', '0.015 (0.031) ', '0.000 (0.000) ']
['0.012 (0.024) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.988 (0.024) ', '0.000 (0.000) ']
['0.017 (0.033) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.983 (0.033) ']
RF-Test
['0.512 (0.025) ', '0.087 (0.031) ', '0.013 (0.025) ', '0.113 (0.047) ', '0.275 (0.031) ']
['0.250 (0.000) ', '0.700 (0.061) ', '0.000 (0.000) ', '0.050 (0.061) ', '0.000 (0.000) ']
['0.133 (0.163) ', '0.133 (0.163) ', '0.000 (0.000) ', '0.200 (0.163) ', '0.533 (0.163) ']
['0.280 (0.098) ', '0.160 (0.080) ', '0.120 (0.098) ', '0.240 (0.196) ', '0.200 (0.000) ']
['0.000 (0.000) ', '0.333 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ', '0.667

In [9]:

df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & 
                                (~df_adapted_combo['EDtype'].str.contains('Others'))]
YTrain_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Train') &
                                (~df_adapted_combo['EDtype'].str.contains('Others'))]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & 
                               (~df_adapted_combo['EDtype'].str.contains('Others'))]
YTest_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Test') & 
                                (~df_adapted_combo['EDtype'].str.contains('Others'))]#.to_numpy()
#GridSearchClassifiers(df_train_eds, YTrain_eds, sampling_strategy)
print(df_train_eds.shape, df_test_eds.shape)
#GridSearch_LVQ(df_train_eds, YTrain_eds, sampling_strategy)

(85, 29) (32, 29)


In [10]:
nan_mean=np.nanmean(df_train_eds.to_numpy(), axis=0)
nan_std=np.nanstd(df_train_eds.to_numpy(), axis=0)
z_train_eds_nan=((df_train_eds.to_numpy()-nan_mean)/nan_std)
z_train_ed_df=pd.DataFrame(data=z_train_eds_nan, columns=adapted_combo_cols)
z_test_ed_df=pd.DataFrame(data=((df_test_eds.to_numpy()-nan_mean)/nan_std), columns=adapted_combo_cols)

kernel_mm= mf.ImputationKernel(data=z_train_ed_df,num_datasets=1,mean_match_candidates=5)
kernel_mm.mice(10)
z_train_eds=pd.DataFrame(data=kernel_mm.complete_data(), columns=adapted_combo_cols)
temp_test=kernel_mm.impute_new_data(z_test_ed_df)
z_test_eds=pd.DataFrame(data=temp_test.complete_data(), columns=adapted_combo_cols)


In [26]:
GridSearchClassifiers(z_train_eds, YTrain_eds, sampling_strategy, param_grid)

RF: 
     n_Trees  Max_Features  Lowest_CWA
11      500            25    0.903788
12      100            29    0.901263
6       100            20    0.901010
10      300            25    0.901010
KNN: 
    K Dist Metric  Lowest_CWA
3  3      cosine    0.676010
4  5      cosine    0.664141
5  7      cosine    0.644697
1  5   minkowski    0.630051
LDA: 
   Solver  Lowest_CWA
2  eigen    0.743687
1   lsqr    0.710101
0    svd    0.701515
3  eigen         NaN
Linear SVM: 
        C  Lowest_CWA
3    1.0    0.756313
4   10.0    0.756313
5  100.0    0.756313
2    0.1    0.726515
RBF SVM: 
         C  Gamma  Lowest_CWA
26   10.0  0.010    0.791667
32  100.0  0.010    0.786111
31  100.0  0.001    0.763131
20    1.0  0.010    0.734596


In [34]:
param_grid_lvq2={'prot_choice': [np.array([3,2,1,1]),np.array([3,3,2,1]),np.array([3,2,2,1]),
            np.array([1,1,1,1]), np.array([1,1,2,1]), np.array([2,1, 2, 1])],
               'n_comp':[7,15, 20,29], 'scorer_name':'Lowest_CWA'}
GridSearch_LVQ(z_train_eds, YTrain_eds, sampling_strategy, param_grid_lvq2)

GMLVQ: 
      Prot per cls Solver  Num Component Activation type  Lowest_CWA
114  [3, 3, 2, 1]  lbfgs              7           swish    0.645455
115  [3, 3, 2, 1]   adam              7           swish    0.644444
98   [3, 2, 1, 1]  lbfgs              7           swish    0.638889
99   [3, 2, 1, 1]   adam              7           swish    0.638889


#### Classifying with optimal hyperparameters

In [13]:
iters=5
pipeRF2=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", min_samples_leaf=5,
                        class_weight='balanced_subsample', n_estimators=500, max_features=25))])
pipeKNN2=Pipeline(steps=[('KNN', KNeighborsClassifier(n_neighbors=3,metric='cosine'))])
pipeLDA2=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'))])
pipeQDA2=Pipeline(steps=[('QDA', QuadraticDiscriminantAnalysis(reg_param=0.1))])
pipeLSVC2=Pipeline(steps=[('LinSVC', SVC(kernel="linear", C=1.00))])
pipeRSVC2=Pipeline(steps=[('RbfSVC', SVC(kernel='rbf', C=10, gamma=0.01))])
pipeLGMLVQ2=Pipeline(steps=[('LGMLVQ', LGMLVQ(distance_type='local-adaptive-squared-euclidean', 
            prototype_n_per_class=np.array([3,2,1,1]), activation_type='swish',
            relevance_n_components=7,relevance_localization="prototypes", solver_type='lbfgs'))])
pipeGMLVQ2=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', 
            prototype_n_per_class=np.array([3,2,1,1]), relevance_regularization=0.001,
            activation_type='swish',relevance_n_components=7, solver_type='lbfgs'))])
pipeGNB2=Pipeline(steps=[('GNB', GaussianNB())])
pipeLogLASSO2=Pipeline(steps=[('LASSO',LogisticRegressionCV(Cs=[0.0001,0.001,0.01,0.1,1], solver='saga',
            class_weight='balanced', cv=5,penalty='l1', scoring=make_scorer(lowest_cwacc)))])

pipeClassifiers2={'RF':pipeRF2, 'KNN':pipeKNN2, 'LDA':pipeLDA2, 'QDA': pipeQDA2, 'LSVC': pipeLSVC2, 
                 'RSVC': pipeRSVC2, 'LGMLVQ': pipeLGMLVQ2, 'GMLVQ': pipeGMLVQ2, 'GNB': pipeGNB2,
                 'LogLASSO': pipeLogLASSO2}
nclasses2=len(np.unique(YTrain_eds))
sm2=SMOTE(sampling_strategy=sampling_strategy)
X_ed,Y_ed=sm2.fit_resample(z_train_eds, YTrain_eds)
labs2=np.unique(YTrain_eds)
print(labs2)
train_conf_eds, test_conf_eds, classifiers_eds={},{},{}

['Anorexia nervosa' 'Binge-ED' 'Bulimia nervosa' 'Other ED']


In [14]:
num_prototypes=np.sum(np.array([1,1, 1]))
num_features, iters=df_train_eds.shape[1],5
all_relevance, all_prots=np.zeros((nclasses2, num_features, iters)), np.zeros((num_prototypes, num_features, iters))

for key, clf in pipeClassifiers2.items():
    tr_cmap=np.zeros((nclasses2, nclasses2, iters))
    te_cmap=np.zeros((nclasses2, nclasses2, iters))
    clf_all=[]
    for iter in range(iters):
        clf.fit(X_ed,Y_ed)
        clf_all.append(clf)
        train_pred_labs_eds=clf.predict(z_train_eds)
        test_pred_labs_eds=clf.predict(z_test_eds)
        tr_cmap[:,:,iter]=confusion_matrix(YTrain_eds, train_pred_labs_eds, normalize='true', labels=labs2)
        te_cmap[:,:,iter]=confusion_matrix(YTest_eds, test_pred_labs_eds, normalize='true', labels=labs2)
    classifiers_eds[key]=clf_all
    train_conf_eds[key], test_conf_eds[key]=tr_cmap, te_cmap
    cmap_dict={'Training': tr_cmap, 'Test':te_cmap}
    for split_set in ['Training', 'Test']:
        print(key+'-'+split_set)    
        if split_set=='Training':
            printlabs=[]
            for lab in labs2:
                if (lab==labs2[2]): # | (lab==labs2[3]):
                    printlabs.append('    '+lab)
                else:
                    printlabs.append(lab)
            print(printlabs)
        mean_glob, std_glob=cmap_dict[split_set].mean(axis=2), cmap_dict[split_set].std(axis=2)
        disp_num_glob=np.reshape(np.stack((mean_glob, std_glob)).ravel('F'),(nclasses2,2*nclasses2))
        for idx in range(nclasses2):
            cmap_cell=[]
            for jdx in range(nclasses2):
                cmap_cell.append('%.3f (%.3f) '%(mean_glob[idx,jdx], std_glob[idx,jdx]))
            print(cmap_cell)    

RF-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED']
['0.886 (0.000) ', '0.029 (0.000) ', '0.029 (0.000) ', '0.057 (0.000) ']
['0.000 (0.000) ', '0.860 (0.020) ', '0.140 (0.020) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ']
['0.235 (0.000) ', '0.059 (0.000) ', '0.000 (0.000) ', '0.706 (0.000) ']
RF-Test
['0.812 (0.000) ', '0.125 (0.000) ', '0.000 (0.000) ', '0.062 (0.000) ']
['0.350 (0.050) ', '0.625 (0.000) ', '0.025 (0.050) ', '0.000 (0.000) ']
['0.667 (0.000) ', '0.333 (0.000) ', '0.000 (0.000) ', '0.000 (0.000) ']
['0.400 (0.000) ', '0.200 (0.000) ', '0.000 (0.000) ', '0.400 (0.000) ']
KNN-Training
['Anorexia nervosa', 'Binge-ED', '    Bulimia nervosa', 'Other ED']
['0.743 (0.000) ', '0.029 (0.000) ', '0.171 (0.000) ', '0.057 (0.000) ']
['0.050 (0.000) ', '0.850 (0.000) ', '0.100 (0.000) ', '0.000 (0.000) ']
['0.000 (0.000) ', '0.000 (0.000) ', '1.000 (0.000) ', '0.000 (0.000) ']
['0.294 (0.000) ', '0.000 (0